In [1]:
import torch
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [5]:
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011


In [4]:
df['year']=pd.DatetimeIndex(df['datetime']).year

In [6]:
df.set_index(df['datetime'],inplace=True)

In [8]:
df.index=pd.to_datetime(df.index)

In [20]:
freq='D'
resampled_df=df.resample(freq).mean()

In [22]:
resampled_df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year
datetime,,,,,,,,,,,,
2011-01-01,spring,0.0,0.0,1.583333,14.110833,18.181250,80.583333,10.749871,13.791667,27.250000,41.041667,2011.0
2011-01-02,spring,0.0,0.0,1.956522,14.902609,17.686957,69.608696,16.652122,5.695652,29.130435,34.826087,2011.0
2011-01-03,spring,0.0,1.0,1.000000,8.050909,9.470227,43.727273,16.636709,5.454545,55.863636,61.318182,2011.0
2011-01-04,spring,0.0,1.0,1.043478,8.200000,10.606087,59.043478,10.739809,4.695652,63.217391,67.913043,2011.0
2011-01-05,spring,0.0,1.0,1.000000,9.305217,11.463478,43.695652,12.522300,3.565217,66.000000,69.565217,2011.0


In [21]:
resampled_df['season'].replace([1.0,2.0,3.0,4.0],['spring','summer','fall','winter'],inplace=True)

In [23]:
sorted_df=pd.get_dummies(resampled_df,columns=['season'])
sorted_df.head()

,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,season_fall,season_spring,season_summer,season_winter
datetime,,,,,,,,,,,,,,,
2011-01-01,0.0,0.0,1.583333,14.110833,18.181250,80.583333,10.749871,13.791667,27.250000,41.041667,2011.0,0,1,0,0
2011-01-02,0.0,0.0,1.956522,14.902609,17.686957,69.608696,16.652122,5.695652,29.130435,34.826087,2011.0,0,1,0,0
2011-01-03,0.0,1.0,1.000000,8.050909,9.470227,43.727273,16.636709,5.454545,55.863636,61.318182,2011.0,0,1,0,0
2011-01-04,0.0,1.0,1.043478,8.200000,10.606087,59.043478,10.739809,4.695652,63.217391,67.913043,2011.0,0,1,0,0
2011-01-05,0.0,1.0,1.000000,9.305217,11.463478,43.695652,12.522300,3.565217,66.000000,69.565217,2011.0,0,1,0,0


In [37]:
sorted_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 719 entries, 2011-01-01 to 2012-12-19
Freq: D
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   holiday        456 non-null    float64
 1   workingday     456 non-null    float64
 2   weather        456 non-null    float64
 3   temp           456 non-null    float64
 4   atemp          456 non-null    float64
 5   humidity       456 non-null    float64
 6   windspeed      456 non-null    float64
 7   casual         456 non-null    float64
 8   registered     456 non-null    float64
 9   count          456 non-null    float64
 10  year           456 non-null    float64
 11  season_fall    719 non-null    uint8  
 12  season_spring  719 non-null    uint8  
 13  season_summer  719 non-null    uint8  
 14  season_winter  719 non-null    uint8  
dtypes: float64(11), uint8(4)
memory usage: 70.2 KB


In [27]:
target=sorted_df['temp']
target.reset_index(drop=True,inplace=True)

In [28]:
y=target.copy()

In [39]:
y.describe()

count    719.000000
mean       2.196106
std        0.961189
min        1.000000
25%        1.000000
50%        3.000000
75%        3.000000
max        3.000000
Name: temp, dtype: float64

In [30]:
y.uniquelace(to_replace=target[target>10],value=int(3),inplace=True)

In [32]:
y.replace(to_replace=target[target>=7],value=int(2),inplace=True)
y.replace(to_replace=target[target<7],value=int(1),inplace=True)

In [38]:
y.fillna(value=1.0,inplace=True)

### Now makinng the custom model 

In [40]:
import torch.nn as nn
import torch.nn.functional as F

In [41]:
input_size=None
output_size=len(y.unique())

In [ ]:
class Net(nn.Module):
    
    def __init__(self,hidden_size,activation_fn='relu',apply_dropout=False):
        
        super(Net,self).__init__()
        
        self.fc1=nn.Linear(input_size,hidden_size)
        self.fc2=nn.Linear(hidden_size,hidden_size)
        self.fc3=nn.Linear(hidden_size,output_size)
        
        self.hidden_size=hidden_size
        self.activation_fn=activation_fn
        
        self.dropout=None
        if apply_dropout:
            self.dropout=nn.Dropout(0.2)
            
        
    def forward(self,x):
        activation_fn=None
        
        if self.activation_fn=='sigmoid':
            activation_fn=F.torch.sigmoid
            
        if self.activation_fn=='tanh':
            activation_fn=F.torch.tanh
            
        if self.activation_fn=='relu':
            activation_fn=torch.relu
            
        x=activation_fn(self.fc1(x))
        x=activation_fn(self.fc2(x))
        
        if self.dropout != None:
            x=self.Dropout(x)
            
        x=self.fc3(x)
        return F.log_softmax(x,dim=-1)
            